# Read File Tool with E2B Sandbox

Demonstration of the read_file tool functionality using E2B sandbox to read files from a cloned repository.

In [1]:
// Import E2B SDK and load environment
import { Sandbox } from "https://esm.sh/e2b@latest";
import * as dotenv from "jsr:@std/dotenv";

// Import the read_file tool
import { readFileTool, handleReadFile } from '../src/tools/read_file.ts';
import { StreamData } from 'npm:ai';



// Load environment variables
await dotenv.load({ export: true });

const apiKey = Deno.env.get("E2B_API_KEY");
if (!apiKey) {
  throw new Error("E2B_API_KEY environment variable is required");
}

console.log("Environment loaded successfully");

Environment loaded successfully


In [2]:
// Create sandbox and clone repository
const sandbox = await Sandbox.create({
    template: "base",
    apiKey: apiKey,
    timeoutMs: 600_000 // 10 minutes
  });
  
  console.log(`Sandbox created with ID: ${sandbox.id}`);
  
  // Clone the Sudoku Solver repository
  const cloneResult = await sandbox.commands.run("git clone https://github.com/Rahimeen-Altaf/Sudoku-Solver-DSA.git /home/user/sudoku-solver");
  console.log("Repository cloned successfully");
  
  // Configure git
  await sandbox.commands.run("git config --global user.email 'test@example.com'");
  await sandbox.commands.run("git config --global user.name 'Test User'");
  
  console.log("Sandbox ready for testing");

  // Set up mock objects for testing
const mockData = new StreamData();
const mockMessages: any[] = [];
const mockAppendToolCallMessage = (msg: any) => [msg];

console.log('Tool definition:', readFileTool);

Sandbox created with ID: undefined
Repository cloned successfully
Sandbox ready for testing
Tool definition: {
  type: "function",
  function: {
    name: "read_file",
    description: "Read the contents of a file from the filesystem.",
    parameters: {
      type: "object",
      properties: {
        path: { type: "string", description: "The file path to read from" }
      },
      required: [ "path" ]
    }
  }
}


In [3]:
// Test error handling by trying to read a non-existent file
const errorToolCall = {
  id: 'test-error',
  func: {
    name: 'read_file',
    arguments: {
      path: '/home/user/sudoku-solver/non-existent-file.txt'
    }
  }
};

console.log('=== Testing Error Handling ===');
const errorResult = await handleReadFile(
  errorToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Error result:', errorResult[0].tool_call_result);

=== Testing Error Handling ===
Error result: {
  error: "path '/home/user/sudoku-solver/non-existent-file.txt' does not exist"
}


In [4]:
// Test reading a Python source file
const solverToolCall = {
  id: 'test-solver',
  func: {
    name: 'read_file',
    arguments: {
      path: '/home/user/sudoku-solver/solver.py'
    }
  }
};

console.log('=== Reading solver.py ===');
const solverResult = await handleReadFile(
  solverToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Solver.py content preview:', solverResult[0].tool_call_result.content.substring(0, 300) + '...');
console.log('Total file size:', solverResult[0].tool_call_result.content.length, 'characters');

=== Reading solver.py ===
Solver.py content preview: import os
import platform

# Clear the console screen based on the operating system
if platform.system() == 'Windows':
    CLEAR = 'cls'
else:
    CLEAR = 'clear'

# Function to load puzzle data from a text file
def load_puzzle_data(size, percentage):
    filename = 'puzzles.txt'
    with open(filen...
Total file size: 5952 characters


In [7]:
// Test reading README.md file from the cloned repository
const readmeToolCall = {
  id: 'test-readme',
  func: {
    name: 'read_file',
    arguments: {
      path: '/home/user/sudoku-solver/README.md'
    }
  }
};

console.log('=== Reading README.md ===');
const readmeResult = await handleReadFile(
  readmeToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('README content preview:', readmeResult[0].tool_call_result.content.substring(0, 200) + '...');

=== Reading README.md ===
README content preview: # Sudoku Solver

The Sudoku Solver project is an advanced Python application designed to efficiently solve Sudoku puzzles. Sudoku is a number puzzle game played on a 9x9 grid, divided into 3x3 subgr...
